In [1]:
import sys
path = '/Users/xiehao/Desktop/workspace/X2AI/sugar_ai/'
if path not in sys.path:
    sys.path.append(path)

import pandas as pd
from DataBuilder.hisugar.crawler import HigSugarCrawler
from DataBuilder.future_bar1d.builder import FutureBar1dBuilder
from DataBuilder.stock_factors.builder import StockFactorsBuilder
from Base import DBFile, DBSQL
from AIBots.SentimentalBot.robot import SentimentalBot
from AIBots.StockBot.robot import StockBot

# 设置参数
start_date = "2024-06-01"
end_date = "2025-06-14"

# HigSugarCrawler(
#     start_date=start_date,
#     end_date=end_date,
#     db=DBFile(),
# ).crawl()

# FutureBar1dBuilder(
#     start_date=start_date,
#     end_date=end_date,
#     db=DBFile(),
# ).build()

# StockFactorsBuilder(
#     start_date=start_date,
#     end_date=end_date,
#     db=DBFile(),
# ).build()

# # AI 研究
# bot = SentimentalBot(db=DBFile(), start_date=start_date, end_date=end_date)
# bot.analyzing()

# bot = StockBot(db=DBFile(), start_date=start_date, end_date=end_date)
# bot.analyzing()

In [2]:
data = DBFile().read_data(
    table="stock_factors",
    filters={"date":[start_date, end_date]},
)

df = data[data["date"]==data["date"].max()]
df.loc[:, ['pe_ttm', 'roe_ttm']] = df.loc[:, ['pe_ttm', 'roe_ttm']].round(3)
df.loc[:, ['net_profit_ttm_yoy', 'net_profit_to_parent_ttm_yoy']] = df.loc[:, ['net_profit_ttm_yoy', 'net_profit_to_parent_ttm_yoy']].round(4)
# 暂时只选择100个股票进行分析
import random
df = df[df["instrument"].isin(random.sample(df["instrument"].unique().tolist(), 100))]
input_data = df.set_index("instrument").drop(columns=["date"]).to_dict(orient="index")
user_prompt = ""
data_templet = """
# 第 {index} 个股票数据
* 股票代码: {instrument}
* pe_ttm: {pe_ttm}
* roe_ttm: {roe_ttm}
* net_profit_ttm_yoy: {net_profit_ttm_yoy}
* net_profit_to_parent_ttm_yoy: {net_profit_to_parent_ttm_yoy}
"""
for i, (instrument, factors) in enumerate(input_data.items()):
    data_str = data_templet.format(
        index=i + 1,
        instrument=instrument,
        pe_ttm=factors.get("pe_ttm", "N/A"),
        roe_ttm=factors.get("roe_ttm", "N/A"),
        net_profit_ttm_yoy=factors.get("net_profit_ttm_yoy", "N/A"),
        net_profit_to_parent_ttm_yoy=factors.get("net_profit_to_parent_ttm_yoy", "N/A"),
    )
    user_prompt += data_str
print(user_prompt)


# 第 1 个股票数据
* 股票代码: 600966.SH
* pe_ttm: 12.743
* roe_ttm: 0.015
* net_profit_ttm_yoy: -0.8349
* net_profit_to_parent_ttm_yoy: -0.8349

# 第 2 个股票数据
* 股票代码: 600981.SH
* pe_ttm: -9.865
* roe_ttm: -0.045
* net_profit_ttm_yoy: nan
* net_profit_to_parent_ttm_yoy: nan

# 第 3 个股票数据
* 股票代码: 600876.SH
* pe_ttm: -0.848
* roe_ttm: -0.193
* net_profit_ttm_yoy: -2.8067
* net_profit_to_parent_ttm_yoy: -3.135

# 第 4 个股票数据
* 股票代码: 601225.SH
* pe_ttm: 0.481
* roe_ttm: 0.215
* net_profit_ttm_yoy: 0.006
* net_profit_to_parent_ttm_yoy: 0.6912

# 第 5 个股票数据
* 股票代码: 601000.SH
* pe_ttm: 3.371
* roe_ttm: 0.084
* net_profit_ttm_yoy: -0.0484
* net_profit_to_parent_ttm_yoy: 0.0068

# 第 6 个股票数据
* 股票代码: 601066.SH
* pe_ttm: 0.988
* roe_ttm: 0.074
* net_profit_ttm_yoy: 0.3423
* net_profit_to_parent_ttm_yoy: 0.3452

# 第 7 个股票数据
* 股票代码: 600866.SH
* pe_ttm: 1.231
* roe_ttm: 0.166
* net_profit_ttm_yoy: 0.4354
* net_profit_to_parent_ttm_yoy: 0.45

# 第 8 个股票数据
* 股票代码: 600676.SH
* pe_ttm: -2.446
* roe_ttm: -0.082
* net_prof

In [3]:
import json
with open("mapping.json", "r") as f:
    mapping = json.load(f)
mapping

FILED = """
## {value}
* 中文名称: {name}
* 投资用法: {describe}
"""

indicators_introducation = ""
for v in mapping:
    field_str = FILED.format(
        value=v["field"],
        name=v["name"],
        describe=v["describe"]
    )
    indicators_introducation += field_str


SYS_PROMPT = """
# 角色
你是一位资深股票分析师，需要基于今日股票池数据进行量化分析，筛选出最具投资价值的10只股票

# 指标介绍

{indicators_introducation}

# 输出格式

输出报告以 json 格式输出，请严格遵守
```
{{
    "instrument": <股票代码>,
    "rank": <投资排名>,
    "reaseon": <推荐理由>,
}}
```

"""
sys_prompt = SYS_PROMPT.format(indicators_introducation=indicators_introducation)


In [4]:
from openai import OpenAI
client = OpenAI(api_key="sk-7e0d7d183ae84e08b8579a537feff921", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": user_prompt},
    ],
    stream=False,
    response_format={'type': 'json_object'}
)
recommend = json.loads(response.choices[0].message.content)

In [5]:
recommend

{'top_stocks': [{'instrument': '002179.SZ',
   'rank': 1,
   'reason': '低市盈率(0.668)和高净资产收益率(0.132)，显示出良好的估值和盈利能力，同时净利润增长率为正(0.0425)，表明公司具有稳定的盈利能力。'},
  {'instrument': '301127.SZ',
   'rank': 2,
   'reason': '较低的市盈率(1.905)和较高的净资产收益率(0.107)，加上正的净利润增长率(0.2563)，显示出公司具有良好的盈利能力和成长性。'},
  {'instrument': '300679.SZ',
   'rank': 3,
   'reason': '极低的市盈率(0.73)和高净资产收益率(0.116)，以及正的净利润增长率(0.2166)，表明公司估值吸引人且盈利能力强。'},
  {'instrument': '002984.SZ',
   'rank': 4,
   'reason': '非常低的市盈率(0.522)和高净资产收益率(0.145)，加上正的净利润增长率(0.2599)，显示出公司具有极高的投资价值。'},
  {'instrument': '002026.SZ',
   'rank': 5,
   'reason': '较低的市盈率(1.703)和正的净利润增长率(0.3387)，显示出公司具有良好的盈利能力和成长性。'},
  {'instrument': '002027.SZ',
   'rank': 6,
   'reason': '适中的市盈率(3.066)和非常高的净资产收益率(0.268)，加上正的净利润增长率(0.0664)，表明公司盈利能力强且稳定。'},
  {'instrument': '002116.SZ',
   'rank': 7,
   'reason': '较低的市盈率(1.549)和较高的净资产收益率(0.117)，加上正的净利润增长率(0.0379)，显示出公司具有良好的盈利能力和稳定性。'},
  {'instrument': '001328.SZ',
   'rank': 8,
   'reason': '较低的市盈率(1.206)和较高的净资产收益率(0.097)，加上正的净利润增长率

In [7]:
result = {}
for i in recommend["top_stocks"]:
    result[i["instrument"]] = {
        "pe_ttm": input_data[i["instrument"]]["pe_ttm"],
        "roe_ttm": input_data[i["instrument"]]["roe_ttm"],
        "net_profit_ttm_yoy": input_data[i["instrument"]]["net_profit_ttm_yoy"],
        "net_profit_to_parent_ttm_yoy": input_data[i["instrument"]]["net_profit_to_parent_ttm_yoy"],
        "rank": i["rank"],
        "reason": i["reason"]
    }
result

{'002179.SZ': {'pe_ttm': 0.668,
  'roe_ttm': 0.132,
  'net_profit_ttm_yoy': 0.0425,
  'net_profit_to_parent_ttm_yoy': 0.1065,
  'rank': 1,
  'reason': '低市盈率(0.668)和高净资产收益率(0.132)，显示出良好的估值和盈利能力，同时净利润增长率为正(0.0425)，表明公司具有稳定的盈利能力。'},
 '301127.SZ': {'pe_ttm': 1.905,
  'roe_ttm': 0.107,
  'net_profit_ttm_yoy': 0.2563,
  'net_profit_to_parent_ttm_yoy': 0.2617,
  'rank': 2,
  'reason': '较低的市盈率(1.905)和较高的净资产收益率(0.107)，加上正的净利润增长率(0.2563)，显示出公司具有良好的盈利能力和成长性。'},
 '300679.SZ': {'pe_ttm': 0.73,
  'roe_ttm': 0.116,
  'net_profit_ttm_yoy': 0.2166,
  'net_profit_to_parent_ttm_yoy': 0.2648,
  'rank': 3,
  'reason': '极低的市盈率(0.73)和高净资产收益率(0.116)，以及正的净利润增长率(0.2166)，表明公司估值吸引人且盈利能力强。'},
 '002984.SZ': {'pe_ttm': 0.522,
  'roe_ttm': 0.145,
  'net_profit_ttm_yoy': 0.2599,
  'net_profit_to_parent_ttm_yoy': 0.2599,
  'rank': 4,
  'reason': '非常低的市盈率(0.522)和高净资产收益率(0.145)，加上正的净利润增长率(0.2599)，显示出公司具有极高的投资价值。'},
 '002026.SZ': {'pe_ttm': 1.703,
  'roe_ttm': 0.071,
  'net_profit_ttm_yoy': 0.3387,
  'net_profit_to_parent

In [8]:
DBFile().save_dict(
    data={end_date: result},
    table="stock_recommend",
)

In [9]:
DBFile().read_dict(
    table="stock_recommend",
)

{Timestamp('2025-06-14 00:00:00'): {'002179.SZ': {'pe_ttm': 0.668,
   'roe_ttm': 0.132,
   'net_profit_ttm_yoy': 0.0425,
   'net_profit_to_parent_ttm_yoy': 0.1065,
   'rank': 1,
   'reason': '低市盈率(0.668)和高净资产收益率(0.132)，显示出良好的估值和盈利能力，同时净利润增长率为正(0.0425)，表明公司具有稳定的盈利能力。'},
  '301127.SZ': {'pe_ttm': 1.905,
   'roe_ttm': 0.107,
   'net_profit_ttm_yoy': 0.2563,
   'net_profit_to_parent_ttm_yoy': 0.2617,
   'rank': 2,
   'reason': '较低的市盈率(1.905)和较高的净资产收益率(0.107)，加上正的净利润增长率(0.2563)，显示出公司具有良好的盈利能力和成长性。'},
  '300679.SZ': {'pe_ttm': 0.73,
   'roe_ttm': 0.116,
   'net_profit_ttm_yoy': 0.2166,
   'net_profit_to_parent_ttm_yoy': 0.2648,
   'rank': 3,
   'reason': '极低的市盈率(0.73)和高净资产收益率(0.116)，以及正的净利润增长率(0.2166)，表明公司估值吸引人且盈利能力强。'},
  '002984.SZ': {'pe_ttm': 0.522,
   'roe_ttm': 0.145,
   'net_profit_ttm_yoy': 0.2599,
   'net_profit_to_parent_ttm_yoy': 0.2599,
   'rank': 4,
   'reason': '非常低的市盈率(0.522)和高净资产收益率(0.145)，加上正的净利润增长率(0.2599)，显示出公司具有极高的投资价值。'},
  '002026.SZ': {'pe_ttm': 1.703,
   'roe_ttm': 0.